In [253]:
import os
import cv2
import numpy as np

datadir_train = "classified/train"
datadir_test = "classified/test"
categories = ['1c','2c', '5c', '10c', '20c', '50c', '1e', '2e']

training_data = []
for category in categories:
    path = os.path.join(datadir_train, category)
    label = categories.index(category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img))
        img_array = cv2.resize(img_array, (500, 500))
        mask = np.zeros(img_array.shape[:2],np.uint8)
        bgdModel = np.zeros((1,65),np.float64)
        fgdModel = np.zeros((1,65),np.float64)
        rect = (50,50,450,290)
        cv2.grabCut(img_array,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
        mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
        img_array = img_array*mask2[:,:,np.newaxis]
        img_array = cv2.resize(img_array, (150,150))
        training_data.append([img_array, label])

In [254]:
import numpy as np
x_train = []
y_train = []

for features, label in training_data:
    x_train.append(features)
    y_train.append(label)
    
x_train = np.array(x_train)

In [255]:
testing_data = []
for category in categories:
    path = os.path.join(datadir_test, category)
    class_num = categories.index(category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img))
        img_array = cv2.resize(img_array, (500, 500))
        mask = np.zeros(img_array.shape[:2],np.uint8)
        bgdModel = np.zeros((1,65),np.float64)
        fgdModel = np.zeros((1,65),np.float64)
        rect = (50,50,450,290)
        cv2.grabCut(img_array,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
        mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
        img_array = img_array*mask2[:,:,np.newaxis]
        img_array = cv2.resize(img_array, (150,150))
        testing_data.append([img_array, class_num])

In [256]:
x_test = []
y_test = []

for features, label in testing_data:
    x_test.append(features)
    y_test.append(label)
    
x_test = np.array(x_test)

In [257]:
from collections import Counter
def k_nearest_neighbors(predict, k):
    distences = []
    for image in training_data:
        distences.append([np.linalg.norm(image[0] - predict), image[1]])
    distences.sort()
    votes = [i[1] for i in distences[:k]]
    votes = ''.join(str(e) for e in votes)
    votes = votes.replace(',', '')
    votes = votes.replace(' ', '')
    result = Counter(votes).most_common(1)[0][0]
    return result

In [258]:
y_test = ''.join(str(e) for e in y_test)

In [263]:
start = time()
correct = 0
total = 0
for image in testing_data:
    prediction = k_nearest_neighbors(image[0], 30)
    if int(prediction) == image[1]:
        correct += 1
    total += 1
accuracy = correct/total
end = time()
print(end-start)

18.849449634552002


In [264]:
print(accuracy)

0.2625


In [ ]:
import sys
import cv2 as cv
import numpy as np

filename = 'src/1e/IMG_4197.JPG'
# Loads an image
src = cv.imread(cv.samples.findFile(filename), cv.IMREAD_COLOR)
# Check if image is loaded fine
if src is None:
    print('Error opening image!')
    print('Usage: hough_circle.py [image_name -- default ' + default_file + '] \n')

In [ ]:
def resize_img(img):
    scale_percent = 500 / img.shape[1]
    w = int(img.shape[1] * scale_percent)
    h = int(img.shape[0] * scale_percent)
    dim = (w, h)
    new_img = cv.resize(img, dim)
    return new_img

In [ ]:
new_img = resize_img(src)

gray = cv.cvtColor(new_img, cv.COLOR_BGR2GRAY)

gray = cv.medianBlur(gray, 5)

rows = gray.shape[0]
circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 1, rows / 8, param1=100, param2=30, minRadius=1, maxRadius=70)

circle_img = new_img
numero_piece = 0
Images = []
if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0,:]:

        radius = i[2]

        img_name = str('piece' + str(numero_piece))
        img_cropped = new_img[i[1] - (radius + 5):i[1] + (radius + 5), i[0] - (radius + 5):i[0] + (radius + 5), ]
        numero_piece += 1
        Images.append(img_cropped)
        #cv.imshow(img_name, img_cropped)
        #cv.imwrite(img_name + '.jpg', img_cropped)

cv.imshow("detected circles", circle_img)

cv.waitKey(0)

In [268]:
somme_de_pieces = 0
for image in Images:
    prediction = k_nearest_neighbors(image[0], 30)
    if prediction == 0:
        somme_de_pieces += 0.01
    if prediction == 1:
        somme_de_pieces += 0.02
    if prediction == 2:
        somme_de_pieces += 0.05
    if prediction == 3:
        somme_de_pieces += 0.10
    if prediction == 4:
        somme_de_pieces += 0.20
    if prediction == 5:
        somme_de_pieces += 0.50
    if prediction == 6:
        somme_de_pieces += 1.00
    if prediction == 7:
        somme_de_pieces += 2.00

SyntaxError: unexpected EOF while parsing (<ipython-input-268-2c607de7a7be>, line 4)

In [ ]:
if prediction == 0:
    print("la valeur de la pièce est de 1 centime")
elif prediction == 1:
    print("la valeur de la pièce est de 2 centimes")
elif prediction == 2:
    print("la valeur de la pièce est de 5 centimes")
elif prediction == 3:
    print("la valeur de la pièce est de 10 centimes")
elif prediction == 4:
    print("la valeur de la pièce est de 20 centimes")
elif prediction == 5:
    print("la valeur de la pièce est de 50 centimes")
elif prediction == 6:
    print("la valeur de la pièce est de 1 euro")
elif prediction == 7:
    print("la valeur de la pièce est de 2 euros")